In [1]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

### Пользовался вот этим руководством по временным рядам 
https://www.tensorflow.org/tutorials/structured_data/time_series?hl=ru#part_1_forecast_a_univariate_time_series

In [2]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [3]:
df = pd.read_csv(csv_path)

In [4]:
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


По задумке прогнозирую погоду, которая будет через 6 часов в будущем, использую 5 дней наблюдений

In [5]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
    """
    history_size  - размер последнего временного интервала
    target_size  - насколько далеко в будущее модель должна научиться прогнозировать    
    """
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, 1)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        labels.append(dataset[i+target_size])
        return np.array(data), np.array(labels)

In [6]:
TRAIN_SPLIT = 300000
tf.random.set_seed(13)

In [7]:
# отбираю один признак - температуру
uni_data = df['T (degC)']
uni_data.index = df['Date Time']
uni_data.head()

Date Time
01.01.2009 00:10:00   -8.02
01.01.2009 00:20:00   -8.41
01.01.2009 00:30:00   -8.51
01.01.2009 00:40:00   -8.31
01.01.2009 00:50:00   -8.27
Name: T (degC), dtype: float64

In [8]:
uni_data = uni_data.values

In [9]:
# стандартизация 
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()

uni_data = (uni_data-uni_train_mean)/uni_train_std

In [10]:
univariate_past_history = 20
univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

In [11]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target temperature to predict')
print (y_train_uni[0])

Single window of past history
[[-1.99766294]
 [-2.04281897]
 [-2.05439744]
 [-2.0312405 ]
 [-2.02660912]
 [-2.00113649]
 [-1.95134907]
 [-1.95134907]
 [-1.98492663]
 [-2.04513467]
 [-2.08334362]
 [-2.09723778]
 [-2.09376424]
 [-2.09144854]
 [-2.07176515]
 [-2.07176515]
 [-2.07639653]
 [-2.08913285]
 [-2.09260639]
 [-2.10418486]]

 Target temperature to predict
-2.1041848598100876


#### LSTM

In [49]:
def save_model_metrics(modelname, val_mse, mse, val_loss, loss, loss_name, iterations, epochs, neurons):
    
    global metrics
    
    model_metrics = pd.DataFrame({
        'model_name': modelname,
        'val_MSE': [val_mse],
        'MSE': [mse],
        'val_loss': [val_loss],
        'loss': [loss],
        'loss_name': [loss_name],
        'Iterations': [iterations],
        'Epochs': [epochs],
        'neurons': [neurons]
    })
    
    metrics = metrics.append(model_metrics)

In [72]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

#### Модель 1

In [73]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
])
simple_lstm_model.compile(optimizer='adam', loss='mae', metrics = 'MSE')

In [74]:
EVALUATION_INTERVAL = 200
EPOCHS = 10

history = simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=50)

Epoch 1/10
200/200 [==============================] - 3s 7ms/step - loss: 0.2029 - MSE: 0.1611 - val_loss: 0.2063 - val_MSE: 0.0426
Epoch 2/10
200/200 [==============================] - 1s 6ms/step - loss: 0.0020 - MSE: 6.4785e-06 - val_loss: 0.2060 - val_MSE: 0.0425
Epoch 3/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0019 - MSE: 5.7489e-06 - val_loss: 0.2051 - val_MSE: 0.0421
Epoch 4/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0016 - MSE: 4.0247e-06 - val_loss: 0.2046 - val_MSE: 0.0419
Epoch 5/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0019 - MSE: 6.1212e-06 - val_loss: 0.2039 - val_MSE: 0.0416
Epoch 6/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0026 - MSE: 1.1655e-05 - val_loss: 0.2029 - val_MSE: 0.0412
Epoch 7/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0034 - MSE: 1.8246e-05 - val_loss: 0.2016 - val_MSE: 0.0406
Epoch 8/10
200/200 [==============================] 

In [75]:
metrics = pd.DataFrame({
        'model_name': 'model 1',
        'val_MSE': [history.history['val_MSE'][-1]],
        'MSE': [history.history['MSE'][-1]],
        'val_loss': [history.history['val_loss'][-1]],
        'loss': [history.history['loss'][-1]],    
        'loss_name': 'mae',
        'Iterations': [EVALUATION_INTERVAL],
        'Epochs': [EPOCHS],
        'neurons': [8]
    })

In [76]:
metrics

,model_name,val_MSE,MSE,val_loss,loss,loss_name,Iterations,Epochs,neurons
0,model 1,0.03958,0.000005,0.198947,0.00184,mae,200,10,8


#### model 2

In [77]:
model_2 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_2.compile(optimizer='adam',
                          loss='binary_crossentropy', metrics = 'MSE')
history = model_2.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=50)

Epoch 1/10
200/200 [==============================] - 3s 7ms/step - loss: -4.4710 - MSE: 5.1379 - val_loss: 0.6836 - val_MSE: 0.0051
Epoch 2/10
200/200 [==============================] - 1s 5ms/step - loss: -11.7722 - MSE: 4.4476 - val_loss: 0.6886 - val_MSE: 0.0076
Epoch 3/10
200/200 [==============================] - 1s 5ms/step - loss: -16.4905 - MSE: 4.4296 - val_loss: 0.6959 - val_MSE: 0.0113
Epoch 4/10
200/200 [==============================] - 1s 5ms/step - loss: -20.7332 - MSE: 4.4279 - val_loss: 0.7056 - val_MSE: 0.0161
Epoch 5/10
200/200 [==============================] - 1s 5ms/step - loss: -24.8073 - MSE: 4.4276 - val_loss: 0.7175 - val_MSE: 0.0220
Epoch 6/10
200/200 [==============================] - 1s 5ms/step - loss: -28.7939 - MSE: 4.4276 - val_loss: 0.7318 - val_MSE: 0.0290
Epoch 7/10
200/200 [==============================] - 1s 5ms/step - loss: -32.7271 - MSE: 4.4276 - val_loss: 0.7484 - val_MSE: 0.0371
Epoch 8/10
200/200 [==============================] - 1s 5ms/st

In [78]:
save_model_metrics('model 2', history.history['val_MSE'][-1], history.history['MSE'][-1],
                  history.history['val_loss'][-1], history.history['loss'][-1],
                  'binary_crossentropy', EVALUATION_INTERVAL, EPOCHS, 8)

In [79]:
metrics

,model_name,val_MSE,MSE,val_loss,loss,loss_name,Iterations,Epochs,neurons
0,model 1,0.039580,0.000005,0.198947,0.001840,mae,200,10,8
0,model 2,0.066991,4.427599,0.812438,-44.352325,binary_crossentropy,200,10,8


#### model 3

In [80]:
model_3 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model_3.compile(optimizer='adam', loss='mae', metrics = 'MSE')
history = model_3.fit(train_univariate, epochs=20,
                      steps_per_epoch=100,
                      validation_data=val_univariate, validation_steps=50)

Epoch 1/20
100/100 [==============================] - 2s 9ms/step - loss: 2.1232 - MSE: 4.5126 - val_loss: 0.2445 - val_MSE: 0.0598
Epoch 2/20
100/100 [==============================] - 1s 6ms/step - loss: 2.1042 - MSE: 4.4278 - val_loss: 0.2420 - val_MSE: 0.0586
Epoch 3/20
100/100 [==============================] - 1s 6ms/step - loss: 2.1042 - MSE: 4.4277 - val_loss: 0.2406 - val_MSE: 0.0579
Epoch 4/20
100/100 [==============================] - 1s 6ms/step - loss: 2.1042 - MSE: 4.4277 - val_loss: 0.2397 - val_MSE: 0.0575
Epoch 5/20
100/100 [==============================] - 1s 6ms/step - loss: 2.1042 - MSE: 4.4276 - val_loss: 0.2391 - val_MSE: 0.0572
Epoch 6/20
100/100 [==============================] - 1s 6ms/step - loss: 2.1042 - MSE: 4.4276 - val_loss: 0.2387 - val_MSE: 0.0570
Epoch 7/20
100/100 [==============================] - 1s 6ms/step - loss: 2.1042 - MSE: 4.4276 - val_loss: 0.2384 - val_MSE: 0.0568
Epoch 8/20
100/100 [==============================] - 1s 6ms/step - loss: 2.

In [81]:
save_model_metrics('model 3', history.history['val_MSE'][-1], history.history['MSE'][-1],
                  history.history['val_loss'][-1], history.history['loss'][-1],
                  'mae', 100, 20, 128)

In [82]:
metrics

,model_name,val_MSE,MSE,val_loss,loss,loss_name,Iterations,Epochs,neurons
0,model 1,0.039580,0.000005,0.198947,0.001840,mae,200,10,8
0,model 2,0.066991,4.427599,0.812438,-44.352325,binary_crossentropy,200,10,8
0,model 3,0.056698,4.427606,0.238113,2.104187,mae,100,20,128


In [83]:
model_4 = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
])
model_4.compile(optimizer='adam', loss='mae', metrics = 'MSE')

history = model_4.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=50)

Epoch 1/10
200/200 [==============================] - 3s 7ms/step - loss: 0.1022 - MSE: 0.0963 - val_loss: 0.1999 - val_MSE: 0.0400
Epoch 2/10
200/200 [==============================] - 1s 6ms/step - loss: 0.0191 - MSE: 5.3411e-04 - val_loss: 0.1808 - val_MSE: 0.0327
Epoch 3/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0123 - MSE: 2.3356e-04 - val_loss: 0.1618 - val_MSE: 0.0262
Epoch 4/10
200/200 [==============================] - 1s 6ms/step - loss: 0.0113 - MSE: 2.0545e-04 - val_loss: 0.1504 - val_MSE: 0.0226
Epoch 5/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0105 - MSE: 1.6448e-04 - val_loss: 0.1294 - val_MSE: 0.0167
Epoch 6/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0106 - MSE: 1.6448e-04 - val_loss: 0.1192 - val_MSE: 0.0142
Epoch 7/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0075 - MSE: 8.7553e-05 - val_loss: 0.1044 - val_MSE: 0.0109
Epoch 8/10
200/200 [==============================] 

In [84]:
save_model_metrics('model 4', history.history['val_MSE'][-1], history.history['MSE'][-1],
                  history.history['val_loss'][-1], history.history['loss'][-1],
                  'mae', 200, 10, 64)

### Вывод

In [85]:
metrics

,model_name,val_MSE,MSE,val_loss,loss,loss_name,Iterations,Epochs,neurons
0,model 1,0.039580,0.000005,0.198947,0.001840,mae,200,10,8
0,model 2,0.066991,4.427599,0.812438,-44.352325,binary_crossentropy,200,10,8
0,model 3,0.056698,4.427606,0.238113,2.104187,mae,100,20,128
0,model 4,0.004022,0.000123,0.063417,0.009157,mae,200,10,64


на данных примерах я выяснил:
-  binary_crossentropy для задач LSTM - не лучший выбор \
-  бездумное увеличение нейронов в слое не даст значительных увеличений метрик
-  лучшие всех себя показали модели 4 и 1